1\. rozglądam sie po systemie, sprawdzam użytkowników, bazy

In [ ]:
use master;
select [name] from master.sys.databases;
select [name] from master.sys.sql_logins;

2\. zakładam bazę

In [ ]:
/*
IF EXISTS (SELECT [name] FROM master.sys.sysdatabases WHERE name = 'DB_OKNO')
    BEGIN
    PRINT 'baza DB_OKNO istnieje';    
    DROP DATABASE DB_OKNO;
    PRINT 'baza usunieta';
    END
*/

DROP DATABASE IF EXISTS DB_OKNO ;
go

CREATE DATABASE DB_OKNO 
/*    ON PRIMARY (
        name=okno,
        filename='/var/opt/mssql/data/okno.mdf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )
    LOG ON (
        name=okno_log,
        filename='/var/opt/mssql/data/okno.ldf',
        size=20MB,
        maxsize=50MB,
        filegrowth=10MB
    )  */
COLLATE POLISH_100_CI_AS_SC_UTF8
GO
    

    

dodaję tabele   \*\*\* SŁOWNIKÓW \*\*\*

In [423]:
use DB_OKNO
GO

/* *** slowniki *** */
DROP TABLE IF EXISTS icon_name;
DROP TABLE IF EXISTS units ;
DROP TABLE IF EXISTS icon;
DROP TABLE IF EXISTS lang;
DROP TABLE IF EXISTS issue;
DROP TABLE IF EXISTS code;
DROP TABLE IF EXISTS category;

go
CREATE TABLE category (
    ID_category DECIMAL(10) NOT NULL ,
    category_name nvarchar(32)

    PRIMARY KEY ( ID_category )
);
go


INSERT INTO category VALUES( 10000 , 'Kosiarki'); 
INSERT INTO category VALUES( 10001 , '- Kosiarki spalinowe'); 
INSERT INTO category VALUES( 10002 , '- Kosiarki elektryczne'); 
INSERT INTO category VALUES( 20000 , 'Ślimacznice'); 



CREATE TABLE code (
    ID_code BIGINT NOT NULL IDENTITY,
    code_string varchar(32),
    EAN CHAR(13), -- po EAN (w tym rozwiazaniu) nie wyszukujemy, są EAN zaczynające się od 0 dlatego CHAR a nie BIGINT/DECIMAL(13)
    active BIT NOT NULL DEFAULT 1,
    category DECIMAL(10), -- kategorie 1.000, 2.000  podkategorie 1.001 1.002 ...   taka notacja jest narzucona przez zewnetrzna bazę

    PRIMARY KEY ( ID_code ),
    FOREIGN KEY ( category ) REFERENCES category( ID_category )
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
go


INSERT INTO code ( code_string ) VALUES ( '80C051' );  
INSERT INTO code ( code_string ) VALUES ( '80C052' );

INSERT INTO code ( code_string ) VALUES ( '0602'); 
INSERT INTO code ( code_string ) VALUES ( '0603');  

INSERT INTO code ( code_string ) VALUES ( 'DED780911');  
INSERT INTO code ( code_string ) VALUES ( 'DED780912');  
INSERT INTO code ( code_string ) VALUES ( 'DED780920');

UPDATE code SET category=10000 WHERE code_string='80C051';  
UPDATE code SET category=10000 WHERE code_string='80C052';  






CREATE TABLE issue (
    ID_issue BIGINT NOT NULL IDENTITY ,
    issue_name varchar(255) NOT NULL 

    PRIMARY KEY ( ID_issue )
);
go


INSERT INTO issue ( issue_name ) VALUES ( '2022.10');
INSERT INTO issue ( issue_name ) VALUES ( '2022.11.Garden');
INSERT INTO issue ( issue_name ) VALUES ( '2023.01');



CREATE TABLE lang (
    ID_lang BIGINT NOT NULL,
    lang_code char(2) NOT NULL,
    currency  nchar(2) NOT NULL,
    zeros   int NOT NULL,
    table_header nvarchar(255),

    PRIMARY KEY ( ID_lang )
);
go



    INSERT INTO lang VALUES (1,'PL','zł',2,'nr. kat.|%|cena|promocja');
    INSERT INTO lang VALUES (2,'CZ','Kč',1,'kat. č.|popis|cena|akční cena');
    INSERT INTO lang VALUES (3,'EN','€',2, 'cat. no.|description|price|promo price');
    INSERT INTO lang VALUES (4,'HU','Ft',0,'cikkszám|leírás|ár|akciós ár');
    -- ****
    INSERT INTO lang VALUES (15,'UA','€',2,'Код|опис|ціна|акційна ціна');





CREATE TABLE icon (
    ID_icon BIGINT NOT NULL IDENTITY,
    icon_name nvarchar(255) NOT NULL UNIQUE,
    -- edit by dept.
    tech  BIT DEFAULT(0),
    promo BIT DEFAULT(0),
    -- target
    paper BIT DEFAULT(0),
    catal BIT DEFAULT(0),
    shipping BIT DEFAULT(0),
    rack BIT DEFAULT(0),

    PRIMARY KEY ( ID_icon )
);
go    

    INSERT INTO icon ( icon_name, tech, rack  )   VALUES ('Moc silnika elektycznego',1 ,1);
    INSERT INTO icon ( icon_name, tech, rack  )   VALUES ('Moc silnika spalinowego' ,1, 1);
    INSERT INTO icon ( icon_name, promo, catal  ) VALUES ('Silnik bezszczotkowy'  ,1, 1);



CREATE TABLE icon_name (
    ID_icon BIGINT NOT NULL,
    ID_lang BIGINT NOT NULL,
    icon_name nvarchar(255) NOT NULL,

    PRIMARY KEY ( ID_icon , ID_lang )
);
go  

INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (1,2,'Výkon elektromotoru');
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (2,2,'Výkon spalovacího motoru' );
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (3,2,'Bezkartáčový motor'    );
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (1,3,'Power of the electric engine');
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (2,3,'Internal combustion engine power' );
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (3,3,'Brushless motor'    );
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (1,15,'Потужність електродвигуна');
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (2,15,'Потужність двигуна внутрішнього згоряння' );
INSERT INTO icon_name ( ID_icon, ID_lang, icon_name  ) VALUES (3,15,'Безщітковий двигун'    );








CREATE TABLE units (
    ID_unit BIGINT NOT NULL IDENTITY,
    unit_code nvarchar(8) NOT NULL UNIQUE,
    unit_name nvarchar(255),    
    multi DECIMAL(24,12)

    PRIMARY KEY ( ID_unit )
);
go    

INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('[ NONE ]','[ *** NONE *** ]' , 1);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('km','kilometr' , 1000);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('m','metr'      , 1);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('dm','decyment' , 0.1);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('cm','centymetr', 0.01);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('mm','milimetr' , 0.001);

INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('MW','megawat' , 1000000);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('kW','kilowat' , 1000);
INSERT INTO units ( unit_code, unit_name, multi  ) VALUES ('W','wat' , 1);




Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.269

In [ ]:
use DB_OKNO;
GO

CREATE OR ALTER FUNCTION dbo.getIdByCode (
    @code_string AS nVARCHAR(32)
) RETURNS BIGINT
AS
BEGIN
    DECLARE @cID BIGINT;
    SET @cID = ( SELECT ID_code FROM dbo.code WHERE code_string = @code_string );
    RETURN (@cID);
END;
go
 
/*
PRINT ( dbo.getIdByCode('80C052') );

DECLARE @cID BIGINT;
SET @cID = ( dbo.getIdByCode('80C052') );
PRINT ( @cID );
*/

tworzę tabelę indd\_name wraz z procedurą

In [ ]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS indd_names;
go

CREATE TABLE indd_names (
    ID_code BIGINT NOT NULL,
    ID_lang BIGINT NOT NULL,

    i_name nvarchar(255),
    i_description nvarchar(max),
    
    PRIMARY KEY ( ID_code , ID_lang )
);
go  


-- INSERT PROCEDURE 
CREATE OR ALTER PROCEDURE dbo.add_names
    @code_string varchar(32),
    @lang_id BIGINT,
    @i_name nvarchar(255),
    @i_description nvarchar(max)
AS
BEGIN
-- PRINT @code_string; PRINT @lang_id;  PRINT @i_name; PRINT @i_description; 

    DECLARE @cID BIGINT = NULL
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NOT NULL 
        BEGIN 
             -- PRINT 'not NULL !';
             -- PRINT  ( CONCAT ( CONCAT ( @code_string , ' : ')  , @cID )) ;
            IF NOT EXISTS ( SELECT ID_code FROM indd_names AS N WHERE ( N.ID_code=@cID AND N.ID_lang = @lang_id) ) 
                BEGIN 
                    -- PRINT 'INSERT';
                    INSERT INTO indd_names ( ID_code , ID_lang , i_name , i_description ) VALUES ( @cID, @lang_id , @i_name , @i_description ); 
                END
            ELSE
                BEGIN 
                    -- PRINT 'UPDATE';
                    UPDATE indd_names  SET  i_name = @i_name , i_description = @i_description  WHERE ( ID_code = @cID AND ID_lang = @lang_id ) ; 
                END 
        END
END
go

In [ ]:
-- uruchamiam procedure wstawiania tresci

USE DB_OKNO;
go

execute dbo.add_names  '0603' , 1 , 'Klucz', 'CRV - first insert' ;
go

execute dbo.add_names  '0603' , 1 , 'Klucz', 'CRV - second insert' ;
go

execute dbo.add_names  '80C051' , 1 , 'Pompa', 'do wody brudnej' ;
go

execute dbo.add_names  '80C051' , 2 , 'Čerpadlo', 'na špinavou vodu' ;
go

execute dbo.add_names  '80C051' , 3 , 'Pump', 'for dirty water' ;
go

execute dbo.add_names  '80C051' , 4 , 'szivattyú', 'piszkos vízhez' ;
go

execute dbo.add_names  '80C051' , 15 , 'Насос', 'для брудної води' ;
go

execute dbo.add_names  ' -- Fake -- ' , 1 , ' ??? ', ' ??? ' ;
go

In [ ]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS prices;
go

CREATE TABLE prices (
    ID_code  BIGINT NOT NULL,
    ID_lang  BIGINT NOT NULL,
    ID_issue BIGINT NOT NULL,

    price DECIMAL(10,2),
    promo DECIMAL(10,2)
    
    PRIMARY KEY ( ID_code , ID_lang , ID_issue )
);
go  


-- INSERT PROCEDURE 
CREATE OR ALTER PROCEDURE dbo.addprices
    @code_string varchar(32),
    @lang_id BIGINT,
    @issue_id BIGINT,    

    @i_price DECIMAL(10,2),
    @i_promo DECIMAL(10,2)
AS
BEGIN
    DECLARE @cID BIGINT = NULL;
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NULL RETURN;   
        INSERT INTO prices ( ID_code , ID_lang , ID_issue, price, promo ) VALUES ( @cID, @lang_id , @issue_id , @i_price, @i_promo );  
END;
go

In [ ]:
use DB_OKNO;
GO

Declare @Issue BIGINT = 2
Declare @Lang BIGINT = 1 -- 1=PL, 2=CZ, 3=EN, 15=UA

EXEC dbo.addprices '80C051' , @Lang , @Issue , 3.21 , 2.99 ;
EXEC dbo.addprices '80C052' , @Lang , @Issue , 4.21 , 3.99 ;
EXEC dbo.addprices '0602' , @Lang , @Issue , 5.21 , 4.99 ;
EXEC dbo.addprices '0603' , @Lang , @Issue , 6.21 , 4.99 ;
EXEC dbo.addprices 'DED780911' , @Lang , @Issue , 7.21 , 5.99 ;
EXEC dbo.addprices 'DED780912' , @Lang , @Issue , 8.21 , 6.99 ;
EXEC dbo.addprices 'DED780920' , @Lang , @Issue , 11.21 , 8.99 ;
 

In [ ]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS product_list;
go

CREATE TABLE product_list (
    ID_code  BIGINT NOT NULL,
    ID_issue BIGINT NOT NULL,

    on_page TINYINT,
    
    PRIMARY KEY ( ID_code , ID_issue )
);
go  


-- INSERT PROCEDURE 
CREATE OR ALTER PROCEDURE dbo.add_product_to_list
    @code_string varchar(32),
    @issue_id BIGINT,    

    @i_page TINYINT
AS
BEGIN
    DECLARE @cID BIGINT = NULL;
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NULL RETURN;   
        INSERT INTO product_list ( ID_code , ID_issue, on_page ) VALUES ( @cID , @issue_id , @i_page );  
END;
go

In [ ]:
use DB_OKNO;
GO

Declare @Issue BIGINT = 2

EXEC dbo.add_product_to_list '80C051'  , @Issue  , 2 ;
EXEC dbo.add_product_to_list '80C052'  , @Issue  , 2 ;
EXEC dbo.add_product_to_list '0602'    , @Issue  , 2 ;
EXEC dbo.add_product_to_list '0603'    , @Issue  , 2 ;
EXEC dbo.add_product_to_list 'DED780911', @Issue , 2 ;
EXEC dbo.add_product_to_list 'DED780912', @Issue , 2 ;
EXEC dbo.add_product_to_list 'DED780920', @Issue , 2 ;

In [ ]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS product_list;
go

CREATE TABLE product_list (
    ID_code  BIGINT NOT NULL,
    ID_issue BIGINT NOT NULL,

    on_page TINYINT,
    
    PRIMARY KEY ( ID_code , ID_issue )
);
go  


-- INSERT PROCEDURE 
CREATE OR ALTER PROCEDURE dbo.add_product_to_list
    @code_string varchar(32),
    @issue_id BIGINT,    

    @i_page TINYINT
AS
BEGIN
    DECLARE @cID BIGINT = NULL;
    SET @cID = ( dbo.getIdByCode( @code_string ));  
    IF @cID IS NULL RETURN;   
        INSERT INTO product_list ( ID_code , ID_issue, on_page ) VALUES ( @cID , @issue_id , @i_page );  
END;
go

In [ ]:
use DB_OKNO;
GO

DROP TABLE IF EXISTS params;
go

CREATE TABLE params (
    ID_code BIGINT NOT NULL,
    ID_icon BIGINT NOT NULL,

    num_value DECIMAL (24,12),
    string_value nVARCHAR(255),
    ID_unit BIGINT NOT NULL
    
    PRIMARY KEY ( ID_code , ID_icon )
    FOREIGN KEY ( ID_code ) REFERENCES dbo.code( ID_code ),
    FOREIGN KEY ( ID_icon ) REFERENCES dbo.icon( ID_icon ),
    FOREIGN KEY ( ID_unit ) REFERENCES dbo.units( ID_unit )

);
go  

INSERT INTO params (ID_code, ID_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C051' ), 1, 120, NULL , 9 );
INSERT INTO params (ID_code, ID_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C052' ), 1, 90, NULL , 9 );
INSERT INTO params (ID_code, ID_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C051' ), 3, NULL, NULL , 1);
INSERT INTO params (ID_code, ID_icon, num_value, string_value, ID_unit) VALUES (   dbo.getIdByCode( '80C052' ), 3, NULL, NULL , 1 );